In [3]:
from pathlib import Path
import pandas as pd
import torch
import io

from IPython.display import clear_output

torch.__version__

'2.5.1'

In [4]:
data_dir = Path(r'F:\cc_data\yt_data')
output_locn = str(data_dir)

# Downloading Videos and Extracting Audio

## Option 1 - With PyTubeFix

In [3]:
from pytubefix import YouTube
from pytubefix.cli import on_progress

In [4]:
import re

def replace_with_underscore(text):
    # Replace all non-alphanumeric characters (including spaces) with underscores
    return re.sub(r'\W+', '_', text)

url_list = ["https://www.youtube.com/watch?v=mQS1tCQUiAM",
            "https://www.youtube.com/watch?v=-OIi7O3cYAg",
            "https://www.youtube.com/watch?v=xutQXlphLiM",
            "https://www.youtube.com/watch?v=j9OHzRypRVs",
            "https://www.youtube.com/watch?v=RFiPUS9-cN0",
            "https://www.youtube.com/watch?v=iCXAT6ZZHuM"
           ]
download_list = []
title_list = []

for url in url_list:
    yt = YouTube(url, on_progress_callback = on_progress)
    
    title = replace_with_underscore(yt.title)
    print(yt.title, title)
    
    video_output = data_dir / f"{title}.mp4"

    try:
        ys = yt.streams.get_highest_resolution()
        ys.download(output_path = data_dir, filename = f"{title}")
        
        ys_audio = yt.streams.get_audio_only()
        ys_audio.download(output_path = data_dir, filename = f"{title}")

        download_list.append(video_output)
        title_list.append(title)

    except:
        print(f'Download failed for {title}')


How To Build A Monster Dividend Portfolio | Ep. 325 How_To_Build_A_Monster_Dividend_Portfolio_Ep_325
I Just Bought $30,000 Of This Stock I_Just_Bought_30_000_Of_This_Stock| 100.0%
We're Off To An Amazing Start We_re_Off_To_An_Amazing_Start███████████| 100.0%
5 Stocks To Buy And Hold Forever 5_Stocks_To_Buy_And_Hold_Forever█████| 100.0%
The Stock Market Has Become Overvalued The_Stock_Market_Has_Become_Overvalued%
I Just Broke A New Record I_Just_Broke_A_New_Record███████████████████| 100.0%


## Option 2 - PyTubeFix and MoviePy


In [5]:
from moviepy.editor import VideoFileClip

for title in title_list:

    video_output = data_dir / f"{title}.mp4"
    
    video_clip = VideoFileClip(str(video_output)) # purely to give us access to methods
    assert video_clip.audio is not None
    
    video_clip.audio.write_audiofile(str(data_dir / f"{title}.mp3"))
    video_clip.close()
    
    clear_output()

NameError: name 'data_dir' is not defined

# Transcribing Audio to Text

In [1]:
import whisper

model = whisper.load_model("turbo")


c:\Users\akhan147\Anaconda3\envs\base11\Lib\site-packages\whisper\__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=dev

In [7]:
for mp3_file in Path(output_locn).glob(f'*.mp3'):
    title = str(mp3_file)
    title = title[title.find('yt_data') +8: title.find('.mp3')]

    print(mp3_file, title)
    result = model.transcribe(str(mp3_file))
    print(result["text"])

    transcription_text = result["text"]

    # Save the result as a txt file
    with open(f"{title}.txt", "w") as txt_file:
        txt_file.write(transcription_text)

F:\cc_data\yt_data\5_Stocks_To_Buy_And_Hold_Forever.mp3 5_Stocks_To_Buy_And_Hold_Forever


c:\Users\akhan147\Anaconda3\envs\base11\Lib\site-packages\whisper\transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 Everyone would like to have a portfolio of companies that we can hold forever. That's the ideal investment, a company that you can buy into one time in your life and watch it grow forever and ever. I consider myself a buy and hold investor, and in my portfolio I'm always looking for companies that I could potentially hold for the rest of my life. After all, holding companies for a long period of time is exactly what we've been instructed to do by the best investors alive. Warren Buffett has famously said that our favorite holding period is forever. Peter Lynch, one of the best stock pickers to ever live, said you only need one or two good stocks a decade. You don't need a lot of action. Now with my stocks, I'm trying to do the same thing, buying companies that I can potentially hold my entire life. But it seems like over time, this is getting more difficult, especially in 2024. The buy and hold strategy is becoming more and more difficult as companies come and go. And part of the reas

In [ ]:
for mp3_file in Path(output_locn).glob(f'*.mp3'):
    title = str(mp3_file)
    title = title[title.find('yt_data') +8: title.find('.mp3')]

    print(mp3_file, title)

F:\cc_data\yt_data\5_Stocks_To_Buy_And_Hold_Forever.mp3 5_Stocks_To_Buy_And_Hold_Forever
F:\cc_data\yt_data\How_To_Build_A_Monster_Dividend_Portfolio_Ep_325.mp3 How_To_Build_A_Monster_Dividend_Portfolio_Ep_325
F:\cc_data\yt_data\I_Just_Bought_30_000_Of_This_Stock.mp3 I_Just_Bought_30_000_Of_This_Stock
F:\cc_data\yt_data\I_Just_Broke_A_New_Record.mp3 I_Just_Broke_A_New_Record
F:\cc_data\yt_data\The_Stock_Market_Has_Become_Overvalued.mp3 The_Stock_Market_Has_Become_Overvalued
F:\cc_data\yt_data\We_re_Off_To_An_Amazing_Start.mp3 We_re_Off_To_An_Amazing_Start


In [7]:
mp3_file_loc = data_dir / f"{title}.m4a"

if mp3_file_loc.exists():
    print("File exists")
else:
    print("File does not exist")

File exists


In [11]:
import speech_recognition as sr
from pydub import AudioSegment
import ffmpeg

def divide_chunks(sound, segment_size_secs=60):
    """ Split audio file into 60s chunks """
    
    segment_size_ms = segment_size_secs*1000
    for start_idx in range(0, len(sound), segment_size_ms):
        # Yield a chunk of audio data from start_idx to start_idx + segment_size_ms
        yield sound[start_idx:start_idx + segment_size_ms]

def transcribe_audio(mp3_file, title, language="en-US"):
    """ Use Speech Recognition API with Google Web Speech API
    to convert audio dialog to text """
    recogniser = sr.Recognizer()      
    transcription_extracts = {}

    print(f"Converting {mp3_file}")

    audio = AudioSegment.from_file(mp3_file)
    # If AudioSegment is not working - e.g. due to broken mp3 headers - we
    # can use ffmpeg as a workaround. However, it's a lot slower.
    # ffmpeg.input(mp3_file).output(wav_file).run() # Convert with ffmpeg
    # print(f"Successfully converted {mp3_file} to {wav_file}.")
    # audio = AudioSegment.from_wav(wav_file) # Read the audio

    segments = list(divide_chunks(audio, segment_size_secs=60)) # split the wav into 60s segments     
    for index, chunk in enumerate(segments):
        with io.BytesIO() as wav_io:
            chunk.export(wav_io, format='wav')
            wav_io.seek(0)  # Move to the start of the BytesIO object before reading from it
                    
            with sr.AudioFile(wav_io) as source:
                audio_data = recogniser.record(source)

            try:
                extracted = recogniser.recognize_google(audio_data, language=language)
                print(f"Chunk {index} extracted.")
                transcription_extracts[index] = extracted
            except sr.UnknownValueError:
                    # Log the unknown value error and continue
                print(f"Chunk {index}: Could not understand the audio. Maybe it was empty.")
        
    print("Extract:")
    for idx, extract in transcription_extracts.items():
        print(f"{idx}: {extract}")

    # Save as JSON file
    with open(data_dir/ f'{title}.json', 'w') as json_file:
        json.dump(transcription_extracts, json_file)

    # Save as TXT file
    with open(data_dir/ f'{title}.txt', 'w') as txt_file:
        for idx, extract in transcription_extracts.items():
            txt_file.write(f"{idx}: {extract}\n")

    # Save as CSV file using pandas
    df = pd.DataFrame(list(transcription_extracts.items()), columns=['Chunk', 'Transcription'])
    df.to_csv(data_dir / f'{title}.csv', index=False)

    clear_output()

In [12]:
for mp3_file in Path(output_locn).glob(f'*.mp3'):
    title = str(mp3_file)
    title = title[title.find('yt_data') +8: title.find('.mp3')]
    transcribe_audio(str(mp3_file), title)

Converting F:\cc_data\yt_data\5_Stocks_To_Buy_And_Hold_Forever.mp3
Chunk 0 extracted.
Chunk 1 extracted.
Chunk 2 extracted.


ConnectionAbortedError: [WinError 10053] An established connection was aborted by the software in your host machine